# **Multi-layer Perceptron**
For classification of Processed Energy Dataset.

In [14]:
# Import the standard tools for pythonic data analysis. 
import csv
import math
import random
import numpy as np 
import pandas as pd 

## Initial import and basic understanding of data

In [15]:
# Import the final dataset as dataframe from csv
df = pd.read_csv('/Users/liangzhengxin/Library/Mobile Documents/com~apple~CloudDocs/Grand Unified Theory/Y3S1 NUS 2022/CS3244 Machine Learning/Project/Datasets/Energy/train/fixed_data_train.csv',sep=',',low_memory=False) # Separate on semicolons

In [16]:
df.head()

,Unnamed: 0,client_id,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,0,train_Client_0,60,11,101,2,31,12,1994,290.568595,...,0,0,0,0,0.0,1,2019,3,19,1
1,1,train_Client_1,69,11,107,2,29,5,2002,202.123247,...,0,0,0,0,0.0,1,2019,4,2,1
2,2,train_Client_10,62,11,301,3,13,3,1986,397.642662,...,0,0,0,0,0.0,1,2019,5,2,1
3,3,train_Client_100,69,11,105,2,7,11,1996,190.591183,...,0,0,0,0,0.0,1,2012,9,25,1
4,4,train_Client_1000,62,11,303,3,14,10,2014,56.083287,...,0,0,0,0,0.0,1,2019,6,17,1


In [17]:
df[df.index == 131452]

,Unnamed: 0,client_id,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
131452,131452,train_Client_96367,60,11,101,2,31,3,2010,103.952853,...,0,0,0,0,0.0,2,2018,11,28,1


In [18]:
df.dtypes # Checking for data types


Unnamed: 0                        int64
client_id                        object
district                          int64
client_catg                       int64
region                            int64
region_group                      int64
creation_date_day                 int64
creation_date_month               int64
creation_date_year                int64
no_months_as_client             float64
services_consumed                 int64
target                            int64
months_of_service               float64
number_of_counter                 int64
number_of_instances               int64
number_of_person_counting         int64
min_reading_remark                int64
max_reading_remark                int64
mean_reading_remark             float64
mean_difference_index           float64
sum_difference_index              int64
min_difference_index              int64
max_difference_index              int64
min_counter_statue                int64
max_counter_statue                int64


## Data Cleaning or Selection

In [19]:
del df['Unnamed: 0'] # Delete Index of CSV file
del df['client_id'] # Delete client_id as it is a string and can't be used as a feature

In [20]:
df.head()

,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,services_consumed,target,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
0,60,11,101,2,31,12,1994,290.568595,1,0,...,0,0,0,0,0.0,1,2019,3,19,1
1,69,11,107,2,29,5,2002,202.123247,1,0,...,0,0,0,0,0.0,1,2019,4,2,1
2,62,11,301,3,13,3,1986,397.642662,1,0,...,0,0,0,0,0.0,1,2019,5,2,1
3,69,11,105,2,7,11,1996,190.591183,1,0,...,0,0,0,0,0.0,1,2012,9,25,1
4,62,11,303,3,14,10,2014,56.083287,1,0,...,0,0,0,0,0.0,1,2019,6,17,1


## Separating Training and Testing Data

Partition the dataset such that both training and testing data have a good mixed of fraud and no fraud data by randomness.

In [21]:
from sklearn.model_selection import train_test_split

# Partion the features from the target to predict
df_features = df[df.columns[df.columns != 'target']].copy() # get columns that are not 'target'; this our features
df_target = df['target'].copy() # get the column named 'target'; this is our label

# (random_state): we use a fixed random seed so we get the same results every time.
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.3, random_state=1) ## RANDOM STATE DETERMINED HERE

print ("Number of training instances: ", len(X_train), "\nNumber of test instances: ", len(X_test), "\nTotal number of instances: ",len(df_target))

Number of training instances:  94840 
Number of test instances:  40647 
Total number of instances:  135487


### Imbalance Data
There $\exists$ imbalance data. Hence, data would be seperated first then undersample

**Training Data**

In [20]:
if False:    
    not_fraud = y_train[y_train == 0].index #Get index of rows that are not fraud
    fraud = y_train[y_train == 1].index # #Get index of rows that are fraud
    not_fraud = np.random.choice(not_fraud, len(fraud), replace=False) # Randomly pick not fraud rows without replacement until the len is the same as fruad
    X_train = pd.concat([X_train.loc[not_fraud],X_train.loc[fraud]]) 
    y_train = pd.concat([y_train.loc[not_fraud],y_train.loc[fraud]])

    print("Training Data", len(X_train),len(y_train))

**Testing Data**

In [21]:
if False:
    not_fraud = y_test[y_test == 0].index #Get index of rows that are not fraud
    fraud = y_test[y_test == 1].index # #Get index of rows that are fraud
    not_fraud = np.random.choice(not_fraud, len(fraud), replace=False) # Randomly pick not fraud rows without replacement until the len is the same as fruad
    X_test = pd.concat([X_test.loc[not_fraud],X_test.loc[fraud]]) 
    y_test = pd.concat([y_test.loc[not_fraud],y_test.loc[fraud]])

    print("Testing Data",len(X_test),len(y_test))

#### Oversampling
Training Data

In [22]:
not_fraud = y_train[y_train == 0].index #Get index of rows that are not fraud
fraud = y_train[y_train == 1].index # #Get index of rows that are fraud
fraud = np.random.choice(fraud, len(not_fraud), replace=True) # Randomly pick not fraud rows without replacement until the len is the same as fruad
X_train = pd.concat([X_train.loc[not_fraud],X_train.loc[fraud]]) 
y_train = pd.concat([y_train.loc[not_fraud],y_train.loc[fraud]])

print("Training Data", len(X_train),len(y_train))

Training Data 179076 179076


Testing Data

In [6]:
not_fraud = y_test[y_test == 0].index #Get index of rows that are not fraud
fraud = y_test[y_test == 1].index # #Get index of rows that are fraud
fraud = np.random.choice(fraud, len(not_fraud), replace=True) # Randomly pick not fraud rows without replacement until the len is the same as fruad
X_test = pd.concat([X_test.loc[not_fraud],X_test.loc[fraud]]) 
y_test = pd.concat([y_test.loc[not_fraud],y_test.loc[fraud]])

print("Testing Data",len(X_test),len(y_test))

Testing Data 76766 76766


Raw Data

In [24]:
## Raw Data
not_fraud = df_target[df_target == 0].index #Get index of rows that are not fraud
fraud = df_target[df_target == 1].index # #Get index of rows that are fraud
fraud = np.random.choice(fraud, len(not_fraud), replace=True) # Randomly pick not fraud rows without replacement until the len is the same as fruad
df_features = pd.concat([df_features.loc[not_fraud],df_features.loc[fraud]]) 
df_target = pd.concat([df_target.loc[not_fraud],df_target.loc[fraud]])

print("Raw Data",len(df_features),len(df_target),flush=True)

Raw Data 255842 255842


In [25]:
y_test.value_counts()

0    38383
1    38383
Name: target, dtype: int64

In [26]:
y_test.head()

89095     0
76482     0
23159     0
110232    0
109412    0
Name: target, dtype: int64

In [27]:
X_test.head()

,district,client_catg,region,region_group,creation_date_day,creation_date_month,creation_date_year,no_months_as_client,services_consumed,months_of_service,...,tally_check_false,sum_tally_value,min_tally_value,max_tally_value,mean_tally_value,counter_type,last_year,last_month,last_day,last_day_is_weekday
89095,62,11,309,3,30,12,1997,258.962196,2,167.592764,...,0,0,0,0,0.0,2,2019,7,30,1
76482,62,11,303,3,1,10,1983,427.704881,2,160.627528,...,0,0,0,0,0.0,2,2019,5,23,1
23159,60,11,101,2,27,6,1985,133.850798,1,4.008296,...,0,0,0,0,0.0,1,1996,8,22,1
110232,60,11,101,2,27,10,1997,258.863632,1,166.048584,...,0,0,0,0,0.0,1,2019,5,24,1
109412,63,11,372,3,30,11,2010,106.121276,1,105.562743,...,0,0,0,0,0.0,1,2019,10,4,1


In [28]:
import json

with open('Feature-Importance.json','r') as jsonfile:
    Feature_Importance = json.load(jsonfile)

Feature_Importance = list(sorted(Feature_Importance.items(),key= lambda x: x[1], reverse=True))
for x in range(len(Feature_Importance)):
    print(Feature_Importance[:x+1])

FileNotFoundError: [Errno 2] No such file or directory: 'Feature-Importance.json'

## Building the MLP Classifier

In [23]:
print("Number of Features: ",len(X_train.columns))

Number of Features:  43


In [24]:
from sklearn.neural_network import MLPClassifier

# MLP = MLPClassifier(hidden_layer_sizes=(35,11), max_iter=500, verbose=1, random_state=1)
MLP = MLPClassifier(hidden_layer_sizes=(50,80,100,70,50,30,20,10,5), max_iter=500, verbose=1, random_state=1)
""" 
MLP = MLPClassifier(hidden_layer_sizes=(50,80,100,70,50,30,20,10,5),    # The ith element represents the number of neurons in the ith hidden layer.
                    activation='relu',                                  # Activation function for the hidden layer.
                    solver='adam',                                      # The solver for weight optimization.
                    alpha=0.0001,                                       # Strength of the L2 regularization term. The L2 regularization term is divided by the sample size when added to the loss.
                    learning_rate='constant',                           # Learning rate schedule for weight updates. Only used when solver='sgd'.
                    learning_rate_init=0.001,                           # The initial learning rate used. It controls the step-size in updating the weights. Only used when solver='sgd' or 'adam'.
                    shuffle=True,                                       # Whether to shuffle samples in each iteration. Only used when solver='sgd' or 'adam'.
                    tol=1e-4,                                           # Tolerance for the optimization.
                    n_iter_no_change=10,                                # Maximum number of epochs to not meet tol improvement                             
                    max_iter=200, 
                    verbose=1, 
                    random_state=1) 
"""


" \nMLP = MLPClassifier(hidden_layer_sizes=(50,80,100,70,50,30,20,10,5),    # The ith element represents the number of neurons in the ith hidden layer.\n                    activation='relu',                                  # Activation function for the hidden layer.\n                    solver='adam',                                      # The solver for weight optimization.\n                    alpha=0.0001,                                       # Strength of the L2 regularization term. The L2 regularization term is divided by the sample size when added to the loss.\n                    learning_rate='constant',                           # Learning rate schedule for weight updates. Only used when solver='sgd'.\n                    learning_rate_init=0.001,                           # The initial learning rate used. It controls the step-size in updating the weights. Only used when solver='sgd' or 'adam'.\n                    shuffle=True,                                       # Wheth

## <font color="orange">Training Starts Here ↓ </font>

### The Basic Method

In [25]:
MLP.fit(X_train, y_train)

Iteration 1, loss = 0.70868847
Iteration 2, loss = 0.69337930
Iteration 3, loss = 0.69322245
Iteration 4, loss = 0.69322410
Iteration 5, loss = 0.69322351
Iteration 6, loss = 0.69322221
Iteration 7, loss = 0.69322379
Iteration 8, loss = 0.69322510
Iteration 9, loss = 0.69322226
Iteration 10, loss = 0.69321981
Iteration 11, loss = 0.69321530
Iteration 12, loss = 0.69322204
Iteration 13, loss = 0.69321605
Iteration 14, loss = 0.69321430
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(50, 80, 100, 70, 50, 30, 20, 10, 5),
              max_iter=500, random_state=1, verbose=1)

### Cross Validation

In [ ]:
""" from statistics import mean
from sklearn.model_selection import cross_val_score
scores = cross_val_score(MLP, X_train, y_train, cv=5)
print(scores)
print(mean(scores)) """

' from statistics import mean\nfrom sklearn.model_selection import cross_val_score\nscores = cross_val_score(MLP, X_train, y_train, cv=5)\nprint(scores)\nprint(mean(scores)) '

In [ ]:
mean(scores)

NameError: name 'mean' is not defined

## Prediction of Model

In [26]:
predicted_target = MLP.predict(X_test)
print(set(predicted_target))
#print("Prediction: ",int(predicted_target))

{1}


## Evaluation Metric

In [27]:
print("Training set score", MLP.score(X_train, y_train))
print("Testing set score", MLP.score(X_test, y_test))

Training set score 0.5
Testing set score 0.05569906758186336


In [28]:
from sklearn import metrics
predicted_target = MLP.predict(X_test)
accuracy = metrics.accuracy_score(y_test, predicted_target)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, predicted_target)
precision = metrics.precision_score(y_test, predicted_target)
recall = metrics.recall_score(y_test, predicted_target)
f1 = metrics.f1_score(y_test, predicted_target)
roc_auc = metrics.roc_auc_score(y_test, predicted_target)
f2 = metrics.fbeta_score(y_test,predicted_target, beta=2.0)

print(f"Accuracy: {accuracy}")
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"ROC AUC: {roc_auc}")
print(f'F2 Score: {f2}')
print(metrics.confusion_matrix(y_test, predicted_target))

Accuracy: 0.05569906758186336
Balanced Accuracy: 0.5
Precision: 0.05569906758186336
Recall: 1.0
F1 score: 0.10552072895061872
ROC AUC: 0.5
F2 Score: 0.22775285194052672
[[    0 38383]
 [    0  2264]]


In [ ]:
y_test.value_counts()

0    38383
1    38383
Name: target, dtype: int64

## Saving of Data

In [ ]:
""" import joblib
joblib.dump(MLP, "model.pkl") 
 """

' import joblib\njoblib.dump(MLP, "model.pkl") \n '

In [ ]:
""" import joblib
MLP = joblib.load("model.pkl") """

' import joblib\nMLP = joblib.load("model.pkl") '

## References
1. https://carpentries-incubator.github.io/machine-learning-novice-sklearn/06-neural-networks/index.html
2. https://scikit-learn.org/stable/modules/neural_networks_supervised.html
3. https://benjaminobi.medium.com/5-minutes-tutorial-on-how-to-compute-and-visualize-the-covariance-matrix-2597ab98d9ee
4. https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html